In [135]:

import time
import pygame
import numpy as np
from swarmae.SwarmAEClient import SwarmAEClient

class TransformStruct:
    """
    存储位姿信息的结构体
    """
    def __init__(self, roll, pitch, yaw, heading, x, y, z):
        self.roll    = roll
        self.pitch   = pitch
        self.yaw     = yaw
        self.heading = heading
        self.x       = x
        self.y       = y

class ImuStruct:
    """
    存储速度信息的结构体
    """
    def __init__(self, vx, vy, vz, v, ax, ay, az, a, g, omega_pitch, omega_roll, omega_yaw):
        self.vx = vx
        self.vy = vy
        self.vz = vz
        self.v  = v
        self.ax = ax
        self.ay = ay
        self.az = az
        self.a  = a
        self.g  = g
        self.omega_pitch = omega_pitch
        self.omega_roll  = omega_roll
        self.omega_yaw   = omega_yaw

class Vehicle:
    """
    用 Vehicle 来统一管理 SwarmAENode 对象
    """
    def __init__(self, Client, vehicle_name, vehicle_no):
        # 设置 Client
        self.Client_ = Client
        
        # 定义车辆基本信息
        self.type_ = "四轮车"       # 节点类型
        self.name_ = vehicle_name  # 节点名称
        self.no_   = vehicle_no    # 节点在队伍中的编号
        
        # 创建车辆节点
        timestamp = int(round(time.time() * 1000))
        _, self.Vehicle_, _ = self.Client_.register_node(node_type = self.type_, 
                                                         node_name = self.name_,
                                                         node_no   = self.no_  ,
                                                         frame_timestamp = timestamp)    
        
        # 读取车辆尺寸
        vehicle_size  = self.Vehicle_.get_size() # 获取车辆尺寸参数
        self.width_   = vehicle_size['y']        # 车辆宽度
        self.length_  = vehicle_size['x']        # 车辆长度
        self.height_  = vehicle_size['z']        # 车辆高度
        
        # 读取车辆颜色
        vehicle_rgb = self.Vehicle_.get_color()  # 读取车辆颜色参数
        self.color_ = np.mat([vehicle_rgb['r'],  # 车辆颜色的 [r, g, b] 值 
                              vehicle_rgb['g'], 
                              vehicle_rgb['b']])
        
        # 读取车辆的其他属性
        _, _, team, _, model, _ = self.Vehicle_.get_node_info()
        self.team_  = team  # 车辆编队归属（分为红/蓝两队） 
        self.model_ = model # 车辆型号
        self.id_    = self.Vehicle_.get_node_id()  # 车辆唯一标识 id
    
    def get_imu_data(self):
        vx, vy, vz, v, ax, ay, az, a, g, p, q, r, _,  = self.Vehicle_.get_velocity()
        ImuData = ImuStruct(vx, vy, vz, v, ax, ay, az, a, g, p, q, r)
        return ImuData  
    
    def get_transform(self):
        yaw, pitch, roll, heading, _ = self.Vehicle_.get_attitude()
        x, y, z, _ = self.Vehicle_.get_location()
        TransformData = TransformStruct(roll, pitch, yaw, heading, x, y, z)
        return TransformData
    
    def control(self, accel, hand_brake, steer):
        # 限制控制量
        # accel > 0 为前进，accel < 0 为减速 / 倒车
        accel = min(accel,  1.0)
        accel = max(accel, -1.0)
        
        if accel != 0:
            hand_brake = 0
        else:
            hand_brake = min(hand_brake,  1.0)
            hand_brake = max(hand_brake, -1.0)
        
        # 向左打轮为负，向右打轮为正
        steer = min(steer,  1.0)
        steer = max(steer, -1.0)
        
        # 发送控制指令
        _, status_steer      = self.Vehicle_.set_vehicle_steer(steer)
        _, status_acc        = self.Vehicle_.control_vehicle(accel)
        _, status_hand_brake = self.Vehicle_.set_vehicle_brake(hand_brake)    
        
        # 返回控制结果
        if ((status_acc        == 200) and 
            (status_steer      == 200) and 
            (status_hand_brake == 200)):
            status_code = 200
        else:
            status_code = 500
            print("\033[0;31;40mConnection Lost!\033[0m")
            
        return status_code
    
# class ManualControl:
#     def __init__(self, Vehicle):
#         self.Vehicle_ = Vehicle
#         self.prev_accel = 0.0
#         self.prev_steer = 0.0
    
    

In [ ]:
AeClient = SwarmAEClient(ue_ip="localhost", ue_port=2000)
Vehicle_1 = Vehicle(Client=AeClient, vehicle_name="UGV_1", vehicle_no=1)

print("Vehilce Info: ")
print("id    : ", Vehicle_1.id_)
print("name  : ", Vehicle_1.name_)
print("N.O.  : ", Vehicle_1.no_)
print("type  : ", Vehicle_1.type_)
print("model : ", Vehicle_1.model_)
print("team  : ", Vehicle_1.team_)
print("color : ", Vehicle_1.color_)
print("width : ", Vehicle_1.width_)
print("length: ", Vehicle_1.length_)
print("height: ", Vehicle_1.height_)

t = 0
while t < 1000:
    status_code = Vehicle_1.control(accel=0.0, hand_brake=0, steer=0.0)
    time.sleep(0.001)
    t = t + 1

# TODO 
# 1. WASD 用来控制车辆前进后退
# 2. 上下左右 用来控制炮管方向
# 3. 空格用来切换车辆
# 4. 绘制车辆传来的传感器数据
# 5. 绘制全局栅格地图
# 6. 栅格地图上绘制路网

vehicle.IMV.TigrM*
Vehilce Info: 
id    :  1
name  :  UGV_1
N.O.  :  1
type  :  四轮车
model :  vehicle.IMV.TigrM*
team  :  None
color :  [[12.28386  12.675285  6.966855]]
width :  1.2999999523162842
length:  3.0
height:  1.5
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1, 0.0]
[0.0, 1

(0.0, 0.0, 0, 1.0)

KeyboardInterrupt: 